# Create lithogenic and biogenic particle fields for Pb scavenging

BLING ends in April, 2019

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from mpl_toolkits.basemap import Basemap
import netCDF4 as nc
import xarray as xr
import cmocean
from scipy.spatial import Delaunay
from scipy import ndimage as nd
from scipy.interpolate import LinearNDInterpolator

%matplotlib inline

#### Parameters / settings:

In [13]:
land_color = '#a9a7a2'

# domain dimensions:
jmin, jmax = 159, 799
imin, imax = 1139, 2179
isize = imax - imin
jsize = jmax - jmin

#### Load files

In [14]:
mesh         = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20220317.nc')
mesh_lon     = mesh['nav_lon'].values
mesh_lat     = mesh['nav_lat'].values
mesh_bathy   = mesh['tmask'][0,:,:,:].values
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
ocean_masked = np.ma.masked_where((mesh_bathy<0.1), mesh_bathy)
depths       = mesh['nav_lev'].values

#### Functions:

In [15]:
def open_file(folder, year, month, return_max=False):   
    # Load interpolated fields:
    with xr.open_dataset(f'{folder}Pb_scavenging_y{year}m{month:02}.nc') as file:
        if return_max:
            return file['biomass'].max(), file['fpop'].max()
        else:
            return file['lithogenic'].values, file['biomass'].values, file['fpop'].values

In [16]:
def save_file(folder, litho_array, bio_array, year, month):   
    
    file_write = xr.Dataset(
        {'lithogenic' : (("deptht","y","x"), litho_array),
         'biogenic' : (("deptht","y","x"), bio_array)}, 
        coords = {
            "time_counter": np.zeros(1),
            "deptht": depths,
            "y": np.zeros(mesh_lat.shape[0]),
            "x": np.zeros(mesh_lon.shape[1])},
    )
    file_write.to_netcdf(f'{folder}Pb_scavenging_y{year}m{month:02}.nc', \
                         unlimited_dims='time_counter')

    return

In [17]:
def create_fields(save_folder, year, month, save=False,
                  alpha=1.0, beta=0.005, exponent=1, litho_max=np.nan, bio_max=np.nan):
    # optional arguments:
    # - alpha     --- fraction of sinking particulate organic matter
    # - beta      --- fraction of lithogenic particle contribution
    # - exponent  --- magnitude of exponent of biogenic particle field (for nonlinear relationship)
    
    # on first call to this function, calculate the maximum values:
    if create_fields.counter == 0:
        print('Start calculating max_bio')
        bio_max = find_max(alpha=alpha)
        print('Max_bio = ', bio_max)
    
    litho, biomass, fpop = open_file('/ocean/brogalla/GEOTRACES/data/Pb-tuning-202210/new-bio-proxy/general-particle-fields/', 
                                     year, month)
    
    litho_field  = litho / litho_max
    sum_bio      = np.add(biomass, alpha*fpop)
    
    bio_field    = (sum_bio / bio_max)**exponent
    
    lithogenic_field_final = litho_field * beta
    biogenic_field_final   = bio_field * (1-beta)
    
    if save:
        save_file(save_folder, lithogenic_field_final, biogenic_field_final, year, month)

    create_fields.counter += 1

    return bio_max

In [18]:
def find_max(alpha=1):
    # use to identify year in which maximum occurs so that moving forward, I only need to load that year
    n = 0
    max_litho_list   = np.empty(len(range(5,9))); 
    max_biomass_list = np.empty(len(range(5,9))); 
    max_fpop_list    = np.empty(len(range(5,9))); 

    # year of maximum biomss
    year = 2012
    for month in range(5,9):
        max_biomass_list[n], max_fpop_list[n] = open_file('/ocean/brogalla/GEOTRACES/data/Pb-tuning-202210/new-bio-proxy/general-particle-fields/', 
                                                            year, month, return_max=True)                
        n+=1
        
    bio_max = np.amax(max_biomass_list + alpha*max_fpop_list)
            
    return bio_max

#### Calculations

Between 2002-2007 (end):
- Lithogenic max in year 2002, month 6 --- 1.633e-7 (index 5)
- Biomass max in year 2004, month 6 --- 6.501e-4 (index 29)
- Fpop max in year 2007, month 6 --- 4.84e-5 (index 65)

Between 2008-2013 (end):
- Lithogenic max in year 2010, month 6 --- 1.833e-7 (index 30)
- Biomass max in year 2012, month 6 --- 6.65e-4 (index 53)
- Fpop max in year 2008, month 6 --- 6.283e-5(index 5) 

Between 2014-2018 (end):
- Lithogenic max in year 2014, month 5 --- 1.657e-7 (index 4)
- Biomass max in year 2016, month 6 --- 6.061e-4 (index 29) 
- Fpop max in year 2015, month 6 --- 5.4775e-5 (index 17)

So, lithogenic maximum occurs in year 2010 and is 1.833e-7, biomass maximum occurs in year 2012 and is 6.65e-4, and fpop maximum is in 2008, but I can probably assume biomass dominates and use year 2012.

In [29]:
save_folder = '/ocean/brogalla/GEOTRACES/data/Pb-tuning-202210/new-bio-proxy/biomass-nonlinear-2-3-fpop-01/'

In [30]:
# Set field creation counter for this set of parameters to zero at the start:
create_fields.counter = 0

bio_maxm = np.nan
for month in range(1,13):   
    bio_maxm = create_fields(f'{save_folder}', 2002, month, save=True,\
                             alpha=0.1, beta=0.005, exponent=2./3., litho_max=1.833e-7, bio_max=bio_maxm)
    
    
# Reset create_fields counter to zero at the end
create_fields.counter = 0

Start calculating max_bio
Max_bio =  0.0006704417417495278
